In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import datetime
import pandas_datareader.data as web
import pandas_datareader as pdr
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from plotly.subplots import make_subplots
from datetime import timedelta

In [2]:
money = 1000000
start = datetime.datetime(2015, 1, 1)
finish = datetime.datetime(2020, 12, 31)
short_window = 30
long_window = 90
stop_loss = 0.05
#symbol = 'SBER.ME'
#symbol = 'BTC-USD'
symbol = 'AAPL'


In [3]:
yf.pdr_override()
df = web.get_data_yahoo(symbol, start=start, end=finish)


[*********************100%***********************]  1 of 1 completed


In [4]:
# Исходные данные
df = yf.download(symbol, start=start, end=finish)
df = df.iloc[:, :-2]
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500
2015-01-05,27.072500,27.162500,26.352501,26.562500
2015-01-06,26.635000,26.857500,26.157499,26.565001
2015-01-07,26.799999,27.049999,26.674999,26.937500
2015-01-08,27.307501,28.037500,27.174999,27.972500
...,...,...,...,...
2020-12-23,132.160004,132.429993,130.779999,130.960007
2020-12-24,131.320007,133.460007,131.100006,131.970001
2020-12-28,133.990005,137.339996,133.509995,136.690002


In [5]:
df_1 = pd.DataFrame()
df_1['Short_window'] = df['Close'].rolling(window=short_window).mean()
df_1['Long_window'] = df['Close'].rolling(window=long_window).mean()
stocks = pd.concat([df,df_1], axis = 1)
stocks

,Open,High,Low,Close,Short_window,Long_window
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,NaN,NaN
2015-01-05,27.072500,27.162500,26.352501,26.562500,NaN,NaN
2015-01-06,26.635000,26.857500,26.157499,26.565001,NaN,NaN
2015-01-07,26.799999,27.049999,26.674999,26.937500,NaN,NaN
2015-01-08,27.307501,28.037500,27.174999,27.972500,NaN,NaN
...,...,...,...,...,...,...
2020-12-23,132.160004,132.429993,130.779999,130.960007,122.093333,118.618527
2020-12-24,131.320007,133.460007,131.100006,131.970001,122.509333,118.800833
2020-12-28,133.990005,137.339996,133.509995,136.690002,123.092000,119.033972


In [6]:
signal = stocks[['Open','Close','Short_window','Long_window']]
signal 

,Open,Close,Short_window,Long_window
Date,,,,
2015-01-02,27.847500,27.332500,NaN,NaN
2015-01-05,27.072500,26.562500,NaN,NaN
2015-01-06,26.635000,26.565001,NaN,NaN
2015-01-07,26.799999,26.937500,NaN,NaN
2015-01-08,27.307501,27.972500,NaN,NaN
...,...,...,...,...
2020-12-23,132.160004,130.960007,122.093333,118.618527
2020-12-24,131.320007,131.970001,122.509333,118.800833
2020-12-28,133.990005,136.690002,123.092000,119.033972


In [7]:
sig = signal.iloc[89:].copy()
sig['Измсрав'] = (sig['Short_window'].ge(sig['Long_window'])).diff()
sig = sig[sig['Измсрав'].fillna(False)].copy()
sig['sign'] = np.where(sig['Short_window'] < sig['Long_window'], 'sig_sale', 'sig_buy')
sig.drop(columns='Измсрав', inplace=True)
sig.head(50)

,Open,Close,Short_window,Long_window,sign
Date,,,,,
2015-07-10,30.485001,30.820000,31.772333,31.787944,sig_sale
2015-11-18,28.940001,29.322500,28.962417,28.927278,sig_buy
2015-12-31,26.752501,26.315001,28.367750,28.434194,sig_sale
2016-04-06,27.557501,27.740000,26.018500,25.924917,sig_buy
2016-05-19,23.660000,23.549999,24.964833,25.067944,sig_sale
2016-08-09,27.057501,27.202499,25.013083,24.915750,sig_buy
2016-12-06,27.375000,27.487499,27.727250,27.738444,sig_sale
2016-12-23,28.897499,29.129999,28.048583,28.007861,sig_buy
2017-07-19,37.619999,37.755001,36.613166,36.651250,sig_sale


In [8]:
pop = sig.drop(['Open', 'Short_window', 'Long_window'], axis=1).reset_index()
lol = signal.drop(['Close', 'Short_window', 'Long_window'], axis=1).reset_index()
lol

,Date,Open
0,2015-01-02,27.847500
1,2015-01-05,27.072500
2,2015-01-06,26.635000
3,2015-01-07,26.799999
4,2015-01-08,27.307501
...,...,...
1505,2020-12-23,132.160004
1506,2020-12-24,131.320007
1507,2020-12-28,133.990005
1508,2020-12-29,138.050003


In [9]:
pip = df.reset_index()

In [10]:
pop = pop.rename(columns={'Close': 'share_price', 'sign': 'Signal'})
lol = lol.rename(columns={'Open': 'share_price'})
sloss = 1 - stop_loss
data = []
for _, row in pop.iterrows():
    data.append({
        'Date': row['Date'],
        'Signal': row['Signal'],
        'share_price': row['share_price'],
    })
    poisk = lol[lol['Date'] == row['Date']]
    if not poisk.empty:
        index = poisk.index[0]
        if (index + 1) < len(lol):
            next_row = lol.iloc[index + 1]
            data.append({
                'Date': next_row['Date'],
                'Signal': 'buy' if row['Signal'] == 'sig_buy' else 'sale',
                'share_price': next_row['share_price']
            })
total = pd.DataFrame(data)
tot = pip.merge(total, how='outer')
tot['limit'] = 0
start = 0
for i in range(len(tot)):
    if i == 0:
        if tot.iloc[i]['Signal'] == 'buy':
            tot.loc[i, 'limit'] = sloss * tot.loc[i, 'share_price'] 
        elif tot.iloc[i]['Signal'] == 'sig_buy':
            tot.loc[i, 'limit'] = 0 
        continue
    if tot.iloc[i]['Signal'] == 'buy':
        start = i
        tot.loc[i, 'limit'] = sloss * tot.loc[i, 'share_price']
    elif tot.loc[i, 'Open'] < tot.loc[i, 'limit']:
        tot.loc[i, 'limit'] = tot.loc[i, 'Open']
    elif tot.iloc[i]['Signal'] == 'sig_buy':
        tot.loc[i, 'limit'] = 0 
    elif tot.iloc[i]['Signal'] == 'sale':
        tot.loc[start:i+1, 'limit'] = tot.loc[start, 'limit'] 
    elif i == len(tot) - 1: 
        tot.loc[start:i+1, 'limit'] = tot.loc[start, 'limit'] 
    else:
        tot.loc[i, 'limit'] = tot.loc[i-1, 'limit'] 
tot['Signal'] = tot.apply(lambda row: 'stop_loss' if (row['Signal'] != 'sig_buy') and (row['limit'] > row['Low'] or row['limit'] > row['Open']) else row['Signal'], axis=1)
conflict_indexes = tot[(tot['Signal'] == 'sig_buy') & (tot['Signal'].shift(-1) == 'stop_loss')].index
while len(conflict_indexes) > 0:
    i = conflict_indexes[0] 
    tot = pd.concat([tot.iloc[:i+2, :], tot.loc[[i+1], :], tot.iloc[i+2:, :]]).reset_index(drop=True)
    tot.loc[i+1, 'Signal'] = 'buy'
    conflict_indexes = tot[(tot['Signal'] == 'sig_buy') & (tot['Signal'].shift(-1) == 'stop_loss')].index

con_in = tot[(tot['Signal'] == 'stop_loss') & (tot['Signal'].shift() == 'sig_sale')].index
for i in con_in:
    tot = pd.concat([tot.iloc[:i+1, :], tot.loc[[i], :], tot.iloc[i+1:, :]]).reset_index(drop=True)
    tot.loc[i, 'Signal'] = 'sale'
tot = tot.dropna(subset=['Signal'])
tot['group'] = (tot['Signal'] == 'sig_buy').cumsum()
def proup(df):
    if 'stop_loss' in df['Signal'].values:
        idx = df[df['Signal'] == 'stop_loss'].index[0]
        df = df.loc[:idx]
    return df
tot = tot.groupby('group').apply(proup).reset_index(drop=True)
tot = tot.reset_index(drop=True)
mask = (tot['Signal'] == 'stop_loss') & (tot['Signal'].shift() == 'sale')
tot = tot[~mask]
first_sig_buy_index = tot[tot['Signal'] == 'sig_buy'].index[0]
tot = tot.loc[first_sig_buy_index:]
tot = tot.reset_index(drop=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  
pd.set_option('display.max_rows', None)
tot.loc[tot['Signal'] == 'stop_loss', 'share_price'] = tot.loc[tot['Signal'] == 'stop_loss', 'limit']
tot = tot.drop(columns=['group'])
for i in range(len(tot)):
    if  tot.loc[i, 'limit'] > tot.loc[i, 'Open']:
        tot.loc[i, 'limit'] = tot.loc[i, 'Open']
tot.loc[tot['Signal'] == 'stop_loss', 'share_price'] = tot.loc[tot['Signal'] == 'stop_loss', 'limit']
tot

,Date,Open,High,Low,Close,Signal,share_price,limit
0,2015-11-18,28.940001,29.372499,28.875000,29.322500,sig_buy,29.322500,0.000000
1,2015-11-19,29.410000,29.937500,29.190001,29.695000,buy,29.410000,27.939500
2,2015-12-14,28.045000,28.170000,27.447500,28.120001,stop_loss,27.939500,27.939500
3,2016-04-06,27.557501,27.745001,27.299999,27.740000,sig_buy,27.740000,0.000000
4,2016-04-07,27.487499,27.605000,27.030001,27.135000,buy,27.487499,26.113124
5,2016-04-26,25.977501,26.325001,25.977501,26.087500,stop_loss,25.977501,25.977501
6,2016-08-09,27.057501,27.235001,27.002501,27.202499,sig_buy,27.202499,0.000000
7,2016-08-10,27.177500,27.225000,26.940001,27.000000,buy,27.177500,25.818625
8,2016-09-09,26.160000,26.430000,25.782499,25.782499,stop_loss,25.818625,25.818625
9,2016-12-23,28.897499,29.129999,28.897499,29.129999,sig_buy,29.129999,0.000000


In [11]:
tot['num_shares'] = 0
tot['share_value'] = 0.0
tot['cash'] = 0.0
tot.loc[0, 'cash'] = money
for i in range(1, len(tot)):
    if tot.loc[i, 'Signal'] == 'buy':
        tot.loc[i, 'num_shares'] = tot.loc[i-1, 'cash'] // tot.loc[i, 'share_price']
        tot.loc[i, 'share_value'] = tot.loc[i, 'num_shares'] * tot.loc[i, 'share_price']
        tot.loc[i, 'cash'] = tot.loc[i-1, 'cash'] - tot.loc[i, 'share_value']

    elif tot.loc[i, 'Signal'] == 'sig_sale':
        tot.loc[i, ['num_shares', 'cash']] = tot.loc[i-1, ['num_shares', 'cash']]

    elif tot.loc[i, 'Signal'] == 'stop_loss':
        tot.loc[i, 'num_shares'] = tot.loc[i-1, 'num_shares']
        tot.loc[i, 'cash'] = (tot.loc[i-1, 'num_shares'] * tot.loc[i, 'share_price']) + tot.loc[i-1, 'cash']
        tot.loc[i, 'share_value'] = tot.loc[i, 'num_shares'] * tot.loc[i, 'share_price']

    elif tot.loc[i, 'Signal'] == 'sale':
        tot.loc[i, 'num_shares'] = tot.loc[i-1, 'num_shares']
        tot.loc[i, 'share_value'] = tot.loc[i, 'num_shares'] * tot.loc[i, 'share_price']
        tot.loc[i, 'cash'] = tot.loc[i, 'share_value'] + tot.loc[i-1, 'cash']

    elif tot.loc[i, 'Signal'] == 'sig_buy':
        tot.loc[i, 'num_shares'] = 0
        tot.loc[i, 'share_value'] = 0
        tot.loc[i, 'cash'] = tot.loc[i-1, 'cash']
tot['cash'] = tot['cash'].round(2)
tot['share_value'] = tot['share_value'].round(2)
otvet = tot[['Date','Signal','num_shares','share_price','share_value','cash']]
otvet

,Date,Signal,num_shares,share_price,share_value,cash
0,2015-11-18,sig_buy,0,29.322500,0.00,1000000.00
1,2015-11-19,buy,34002,29.410000,999998.81,1.19
2,2015-12-14,stop_loss,34002,27.939500,949998.87,950000.06
3,2016-04-06,sig_buy,0,27.740000,0.00,950000.06
4,2016-04-07,buy,34561,27.487499,949995.46,4.60
5,2016-04-26,stop_loss,34561,25.977501,897808.41,897813.01
6,2016-08-09,sig_buy,0,27.202499,0.00,897813.01
7,2016-08-10,buy,33035,27.177500,897808.70,4.30
8,2016-09-09,stop_loss,33035,25.818625,852918.27,852922.57
9,2016-12-23,sig_buy,0,29.129999,0.00,852922.57


In [12]:
cska = tot.set_index('Date')
cska

,Open,High,Low,Close,Signal,share_price,limit,num_shares,share_value,cash
Date,,,,,,,,,,
2015-11-18,28.940001,29.372499,28.875000,29.322500,sig_buy,29.322500,0.000000,0,0.00,1000000.00
2015-11-19,29.410000,29.937500,29.190001,29.695000,buy,29.410000,27.939500,34002,999998.81,1.19
2015-12-14,28.045000,28.170000,27.447500,28.120001,stop_loss,27.939500,27.939500,34002,949998.87,950000.06
2016-04-06,27.557501,27.745001,27.299999,27.740000,sig_buy,27.740000,0.000000,0,0.00,950000.06
2016-04-07,27.487499,27.605000,27.030001,27.135000,buy,27.487499,26.113124,34561,949995.46,4.60
2016-04-26,25.977501,26.325001,25.977501,26.087500,stop_loss,25.977501,25.977501,34561,897808.41,897813.01
2016-08-09,27.057501,27.235001,27.002501,27.202499,sig_buy,27.202499,0.000000,0,0.00,897813.01
2016-08-10,27.177500,27.225000,26.940001,27.000000,buy,27.177500,25.818625,33035,897808.70,4.30
2016-09-09,26.160000,26.430000,25.782499,25.782499,stop_loss,25.818625,25.818625,33035,852918.27,852922.57


# Вывод ответов 

### Итоговый дата фрейм

In [13]:
display(otvet)

,Date,Signal,num_shares,share_price,share_value,cash
0,2015-11-18,sig_buy,0,29.322500,0.00,1000000.00
1,2015-11-19,buy,34002,29.410000,999998.81,1.19
2,2015-12-14,stop_loss,34002,27.939500,949998.87,950000.06
3,2016-04-06,sig_buy,0,27.740000,0.00,950000.06
4,2016-04-07,buy,34561,27.487499,949995.46,4.60
5,2016-04-26,stop_loss,34561,25.977501,897808.41,897813.01
6,2016-08-09,sig_buy,0,27.202499,0.00,897813.01
7,2016-08-10,buy,33035,27.177500,897808.70,4.30
8,2016-09-09,stop_loss,33035,25.818625,852918.27,852922.57
9,2016-12-23,sig_buy,0,29.129999,0.00,852922.57


# Вывод ответов о приросте 

In [14]:
last_sale_index = tot[(tot['Signal'] == 'sale')| (tot['Signal'] == 'stop_loss')].last_valid_index()
cas = tot.loc[last_sale_index, 'cash']
growth = ((cas - money) / money) * 100
print('Сумма по последней продаже', cas)
print('Проецент прироста', growth)

Сумма по последней продаже 1307625.64
Проецент прироста 30.76256399999999


# Визуализация

In [15]:
trace1 = go.Ohlc(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])
trace2 = go.Scatter(x=stocks.index, y=stocks['Short_window'], mode='lines', name='Скользящее_30')
trace3 = go.Scatter(x=stocks.index, y=stocks['Long_window'], mode='lines', name='Скользящее_90')

markers_buy = cska[cska['Signal'] == 'buy']
markers_sale = cska[cska['Signal'] == 'sale']
markers_stop = cska[cska['Signal'] == 'stop_loss']

trace4 = go.Scatter(x=markers_buy.index, y=markers_buy['share_price'], mode='markers', name='Buy',
                    marker=dict(color='green', symbol='triangle-up', size=20))
trace5 = go.Scatter(x=markers_sale.index, y=markers_sale['share_price'], mode='markers', name='Sale',
                    marker=dict(color='red', symbol='triangle-down', size=20))
trace6 = go.Scatter(x=markers_stop.index, y=markers_stop['share_price'], mode='markers', name='Stop Loss',
                    marker=dict(color='black', symbol='triangle-down', size=20))
data = [trace1, trace2, trace3,trace4, trace5, trace6]
layout = go.Layout(
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ),
        type='date'
    ),
    yaxis=dict(
        scaleanchor = "x",
        scaleratio = 1,
    )
)
fig = go.Figure(data=data, layout=layout)
fig.show()

fig = make_subplots(rows=2, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.18, 
                    subplot_titles=('Позиции', ''),
                    row_heights=[1, 1])

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)
fig.add_trace(trace3, row=1, col=1)

fig.add_trace(trace4, row=1, col=1)
fig.add_trace(trace5, row=1, col=1)
fig.add_trace(trace6, row=1, col=1)

position = [1 if sig=='buy' else 0 for sig in cska['Signal']]
shag = [val for val in position for _ in (0, 1)]
index = [date for date in cska.index for _ in (0, 1)][1:] + [cska.index[-1]]
first_date = cska.index[0] - timedelta(days=1)
last_date = cska.index[-1] + timedelta(days=1)
index = [first_date] + index + [last_date]
shag = [0] + shag + [shag[-1]]
trace7 = go.Scatter(x=index, y=shag, mode='lines', name='Position', line=dict(color='green'))

fig.add_trace(trace7, row=2, col=1)
fig.show()

